In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
#import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta

In [ ]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [ ]:
import pickle
with open('dict_corrected_DH.obj', 'rb') as handle:
    dict_corrected_DH = pickle.load(handle)
with open('geocoded_locs.obj', 'rb') as handle:
    geocoded_locs = pickle.load(handle)

with open('corpus_mails_DH.obj', 'rb') as handle:
    corpus_mails_DH = pickle.load(handle)

with open('dict_noms_mails_debut_fin.obj', 'rb') as handle:
    dict_noms_mails_debut_fin = pickle.load(handle)
    

with open('liste_de_tous_les_mails.obj', 'rb') as handle:
    liste_de_tous_les_mails = pickle.load(handle)


In [ ]:

    
print str(dict_corrected_DH)[:1000],"\n"
print str(geocoded_locs)[:1000],"\n"
print str(corpus_mails_DH)[:1000],"\n"
print str(dict_noms_mails_debut_fin)[:1000],"\n"
print str(liste_de_tous_les_mails)[:1000],"\n"


In [ ]:
from datetime import timedelta
import datetime
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
title = "DH_studies_map"
my_nodesdict = {}
my_nodes= []
my_edges = []


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'



###FOR NODES
print len(dict_noms_mails_debut_fin)
print dict_noms_mails_debut_fin['Claire Clivaz']
#claire.clivaz@unil.ch

In [ ]:
cnt=0
cnt_not_in_dict=0
for maill in liste_de_tous_les_mails:
    trouve=0
    #print maill
    for y in dict_noms_mails_debut_fin :
        for i,arr in enumerate(dict_noms_mails_debut_fin[y][0]):
            #rint "y", dict_noms_mails_debut_fin[y][0][i]
            
            if isinstance(dict_noms_mails_debut_fin[y][0][i], list) == True:
                print "!!!!!!list_in_arra",dict_noms_mails_debut_fin[y][0][i]
            else:
                if dict_noms_mails_debut_fin[y][0][i] == maill :
                        cnt+=1
                        trouve +=1
                        lat=None
                        lng=None


                        print "mail",maill.lower()
                        print "start",dict_noms_mails_debut_fin[y][0][i+1]
                        print "end",dict_noms_mails_debut_fin[y][0][i+2]
                        print "name",y.lower()



                        mail_address_suffix=dict_noms_mails_debut_fin[y][0][i].lower().split("@",1)[1]
                        print mail_address_suffix
                        lat=dict_corrected_DH[mail_address_suffix.lower()]["geometry"]["location"]["lat"]
                        lng=dict_corrected_DH[mail_address_suffix.lower()]["geometry"]["location"]["lng"]
                        print "lat/lng ",lat," ",lng
                        print "________________"

                        try: 
                #print start
                #print end
                            node = {
                            "id" : maill.lower(),
                            "name" : y.lower(),
                           "lat" : float(lat),
                           "lng" : float(lng),
                           "weight" : float(4),
                            "start" : dict_noms_mails_debut_fin[y][0][i+1].isoformat(),

                           "end" : (dict_noms_mails_debut_fin[y][0][i+2]+timedelta(days=1)).isoformat()
                            }
                        except AttributeError as e:
                            print "STARTSTART"
                            #print datetime.datetime(start[:-5])
                            print e

                            node = {
                                        "id" : maill.lower(),
                                        "name" : y.lower(),
                                       "lat" : float(lat),
                                       "lng" : float(lng),
                                       "weight" : float(4),
                                        "start" : datetime.datetime(2012, 3, 2, 15, 19, 10).isoformat(),

                                       "end" : datetime.datetime(2018, 3, 2, 15, 19, 10).isoformat()
                                        }
                            print node
                        my_nodes.append({ "data" : node })   
                        my_nodesdict[node["id"]]= { "data" : node }
                    
    #SI LE MAIL N EST PAS DANS LE DICT DE NOMS(DONC QUE CE N EST PAS UN EXPEDITEUR)
    if trouve == 0:
        lat=None
        lng=None
        print "MAIL NOT IN NAME DICT: ", maill,"\n_________________"
        cnt_not_in_dict+=1
        mail_address_suffix=maill.lower().split("@",1)[1]
        print mail_address_suffix
        lat=dict_corrected_DH[mail_address_suffix.lower()]["geometry"]["location"]["lat"]
        lng=dict_corrected_DH[mail_address_suffix.lower()]["geometry"]["location"]["lng"]
        print "lat/lng ",lat," ",lng
        print "________________"
                    
        try: 
            #print start
            #print end
                        node = {
                        "id" : dict_noms_mails_debut_fin[y][0][i].lower(),
                        "name" : y.lower(),
                       "lat" : float(lat),
                       "lng" : float(lng),
                       "weight" : float(4),
                        "start" : datetime.datetime(2012, 3, 2, 15, 19, 10).isoformat(),

                       "end" : datetime.datetime(2018, 3, 2, 15, 19, 10).isoformat()
                        }
        except AttributeError as e:
                        print "STARTSTART2"
                        print e

                        node = {
                                    "id" : maill.lower(),
                                    "name" : maill.lower(),
                                   "lat" : float(lat),
                                   "lng" : float(lng),
                                   "weight" : float(4),
                                    "start" : datetime.datetime(2012, 3, 2, 15, 19, 10).isoformat(),

                                   "end" : datetime.datetime(2018, 3, 2, 15, 19, 10).isoformat()
                                    }
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        print node,"\n----------------\n"

        
        
        
        
        
        
        
        
        
        
        
        
    
    
print len(my_nodesdict)
    
                    
                    
                    
                
print "<<<<<<<<<<<COUNT NODES>>>>>>>>>>>>>",cnt
print "<<<<<<COUNT NODES NOT IN DICT>>>>>>",cnt_not_in_dict
print "<<<<<<<<<TOTAL>>>>>>>",cnt_not_in_dict+cnt


    
print len(my_nodesdict)
    
        
    
    
    

In [ ]:
print len(my_nodesdict)

In [ ]:
print  my_nodesdict

In [ ]:
import random
def my_edges_list_singled() :
    my_edges=[]
    extra_nodes_non_autor_list=[]
    print  corpus_mails_DH["./DHFRsample/2017-06/msg00011.html"]
    for mail in corpus_mails_DH:
        #print mail
        #print  corpus_mails_DH[mail]
        if "liste_CCs" in corpus_mails_DH[mail]:
            for maild in corpus_mails_DH[mail]["liste_CCs"] :
                if maild not in dict_noms_mails_debut_fin:
                    if maild not in extra_nodes_non_autor_list:
                        print "OHOH NO, NO maild in liste_mail", maild
                        extra_nodes_non_autor_list.append(maild)
                        print corpus_mails_DH[mail]["sujet_du_message"]
                        if "sujet_du_message" in corpus_mails_DH[mail]:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"].lower(),
                        "target" : maild.lower(),
                        "weight" : float(2),
                        "notes" : unicode(corpus_mails_DH[mail]["sujet_du_message"]),
                        "start": corpus_mails_DH[mail]["date"].isoformat()
                        
                    }
                        else:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"].lower(),
                        "target" : maild.lower(),
                        "weight" : float(2),
                        "start": corpus_mails_DH[mail]["date"].isoformat()
                        
                       
                    }
                            
                my_edges.append({ "data" : edge })

        for maild in corpus_mails_DH[mail]["liste_mails_dests"] :
            if maild not in dict_noms_mails_debut_fin:
                    if maild not in extra_nodes_non_autor_list:
                        #print "OHOH NO, NO maild in liste_mail", maild
                        extra_nodes_non_autor_list.append(maild)
            
                        if "sujet_du_message" in corpus_mails_DH[mail]:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"].lower(),
                        "target" : maild.lower(),
                        "weight" : float(2),
                        "notes" : unicode(corpus_mails_DH[mail]["sujet_du_message"]),
                        "start": corpus_mails_DH[mail]["date"].isoformat()
                        
                        
                    }
                        else:
                            edge = {
                        "id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"])+" "+corpus_mails_DH[mail]["mail_auteur"]+maild+str(float(random.random())),
                        "source" : corpus_mails_DH[mail]["mail_auteur"].lower(),
                        "target" : maild.lower(),
                        "weight" : float(2),
                        "start": corpus_mails_DH[mail]["date"].isoformat()
                        
                       
                    }
            
            my_edges.append({ "data" : edge })



    #print len(my_edges)
    #print extra_nodes_non_autor_list
    #print len(extra_nodes_non_autor_list)
    return [my_edges,extra_nodes_non_autor_list]

my_edges_singled=my_edges_list_singled()[0]
extra_nodes=my_edges_list_singled()[1]

In [ ]:
#print extra_nodes


for mail in extra_nodes:
    
        print mail
        
#        start= mail_array[1]
#        end= mail_array[2]
        mail_address_suffix=mail.split("@",1)[1]
        print mail_address_suffix
        if mail_address_suffix in dict_corrected_DH  :
            
            lat=dict_corrected_DH[mail_address_suffix]["geometry"]["location"]["lat"]
            lng=dict_corrected_DH[mail_address_suffix]["geometry"]["location"]["lng"]
            node = {
                        "id" : mail,
                        "name" : mail,
                       "lat" : float(lat),
                       "lng" : float(lng),
                       "weight" : float(4),

                        }
        else: 
            
        
            node = {
                        "id" : mail,
                        "name" : mail,
                       "weight" : float(4),
                        
                        }
        
            print node
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
    
print len(my_nodesdict)
    
        
    
    
    


In [ ]:
my_edges=my_edges_singled
print len(my_edges)
print my_edges

In [ ]:
def my_edges_list_by_source_dest() :
    my_edges=[]

    for mail in corpus_mails_DH:
        #print mail
         #print  corpus_mails_DH[mail]
        if "liste_CCs" in corpus_mails_DH[mail]:
            for maild in corpus_mails_DH[mail]["liste_CCs"] :
                #print unicode(corpus_mails_DH[mail]["sujet_du_message"])
                edge = {
                    #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                    "source" : corpus_mails_DH[mail]["mail_auteur"],
                    "target" : maild,
                    "weight" : float(2),
                    #"name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])+" "+
                }
                my_edges.append({ "data" : edge })

        for maild in corpus_mails_DH[mail]["liste_mails_dests"] :
            edge = {
                #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                "source" : corpus_mails_DH[mail]["mail_auteur"],
                "target" : maild,
                "weight" : float(5),
                #"name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
            }
            my_edges.append({ "data" : edge })



    print len(my_edges)
    return my_edges

my_edges_duped=my_edges_list_by_source_dest()

In [ ]:
def my_edges_list_with_subject() :
    my_edges=[]

    for mail in corpus_mails_DH:
        #print mail
         #print  corpus_mails_DH[mail]
        if "liste_CCs" in corpus_mails_DH[mail]:
            for maild in corpus_mails_DH[mail]["liste_CCs"] :
                #print unicode(corpus_mails_DH[mail]["sujet_du_message"])
                edge = {
                    #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                    "source" : corpus_mails_DH[mail]["mail_auteur"],
                    "target" : maild,
                    #"weight" : float(2),
                    "name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
                }
                my_edges.append({ "data" : edge })

        for maild in corpus_mails_DH[mail]["liste_mails_dests"] :
            edge = {
                #"id":unicode(corpus_mails_DH[mail]["ref_physique_de_l_article"]),
                "source" : corpus_mails_DH[mail]["mail_auteur"],
                "target" : maild,
                #"weight" : float(5),
                "name" : unicode(corpus_mails_DH[mail]["sujet_du_message"])
            }
            my_edges.append({ "data" : edge })



    print len(my_edges)

    return my_edges

my_edges_duped_with_subject=my_edges_list_with_subject()

In [ ]:


#WE HAVE TO TEST THE EDGES, BUT IT IS AN ARRAY OF DICTS,SO WE NEED TO FLATTEN THEM TO COUNT THEM SIMPLY

import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)




In [ ]:
def list_flattener_and_finder(edges_list_singled,edges_list_duped):
    from collections import Counter
    import ast
    my_edges_replacement_deduped=[]
    receiver=[]
    for i,dic in enumerate(edges_list_duped):
        #print my_edges[i]
        dicto=flatten(edges_list_duped[i])
    #    print dicto
        receiver.append(str(dicto))

    #print receiver
    CREE= Counter(receiver).iteritems()
    total=0
    dups=0
    dups_cnt=0
    
    print edges_list_singled[:1]
    for key, value in CREE:
        #print key
        #print value
        keyB={'data':{}}
        total+=1
        
        #print ast.literal_eval(key)["data_source"]
        #print ast.literal_eval(key)["data_target"]        
        keyB={'data':{"source":ast.literal_eval(key)["data_source"],"target": ast.literal_eval(key)["data_target"]}}
        #print "SUP",key, value
        dups+=value
        dups_cnt+=1
        #print ">>",keyB
        #print ast.literal_eval(key)["data_weight"]    
        keyB["data"]["weight"]= float(value*float(ast.literal_eval(key)["data_weight"]))
        #print "KEYB____",keyB
        my_edges_replacement_deduped.append(keyB)


            





    print "total",total

    print "dups",dups
    print "dups_cnt",dups_cnt
    return my_edges_replacement_deduped

listor=list_flattener_and_finder(my_edges,my_edges_duped)
print listor

In [ ]:
res=dups/dups_cnt



In [ ]:
my_edges=listor

In [ ]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)
my_nodesdictList = []
for key, value in my_nodesdict.iteritems():
    
    my_nodesdictList.append(value)



In [ ]:
#print my_edges
list_nodes_id=[] 
for nodes in  my_nodesdictList:
    if nodes['data']['id'] not in list_nodes_id:
        list_nodes_id.append(nodes['data']['id'])
    
print len(list_nodes_id)

print list_nodes_id
for edge in my_edges:
    source= edge['data']['source']
    target= edge['data']['target']
    if source not in list_nodes_id:
        print "NO!"
        print source
    if target not in list_nodes_id:
        print "NO2!"
        print target


In [ ]:
TOPOGRAM_URL = "http://localhost:3000"

In [ ]:
TOPOGRAM_URL = "https://app.topogram.io"

In [ ]:
title = "alphaDH25"
create_topogram(title, my_nodesdictList, my_edges)